In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "1H"
max_min = 45
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_1H_30_45.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [6]:
all_events_df[all_events_df.eventSec > 60*45].eventSec.mean() / 60

46.20638685694617

In [7]:
all_events_df[all_events_df.eventSec > 60*45].eventSec.median() / 60

45.902613025

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [11]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [12]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.037037,0.135802,0.135802,0.691358
10531---2499736,0.000000,0.090909,0.090909,0.818182
10531---2499741,0.052632,0.122807,0.122807,0.701754
10531---2499757,0.054545,0.109091,0.090909,0.745455
10531---2499766,0.000000,0.157895,0.157895,0.684211
...,...,...,...,...
756---2565880,0.000000,0.238095,0.238095,0.523810
756---2565892,0.015625,0.109375,0.125000,0.750000
756---2565904,0.000000,0.179104,0.164179,0.656716


In [13]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [14]:
motif_stats

,mean,std
Motif,,
ABAB,0.036005,0.029747
ABCA,0.110188,0.037538
ABCB,0.110498,0.037078
ABCD,0.743309,0.080920


# Feature engineering - Shot DF

In [15]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-15-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [16]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-16-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [17]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-17-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [18]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [19]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499736,1,0,1,0
10531---2499741,2,0,2,0
10531---2499757,3,0,3,0
10531---2499766,1,0,1,0
10531---2499777,3,1,1,1
...,...,...,...,...
756---2565880,7,3,2,2
756---2565892,2,2,0,0
756---2565904,2,0,0,2


In [20]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [21]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499736,1,0.000000,1.000000,0.000000
10531---2499741,2,0.000000,1.000000,0.000000
10531---2499757,3,0.000000,1.000000,0.000000
10531---2499766,1,0.000000,1.000000,0.000000
10531---2499777,3,0.333333,0.333333,0.333333
...,...,...,...,...
756---2565880,7,0.428571,0.285714,0.285714
756---2565892,2,1.000000,0.000000,0.000000
756---2565904,2,0.000000,0.000000,1.000000


# Feature engineering - Pass DF - Pass Types

In [22]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-22-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [23]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [24]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
7,8,Simple pass,[{'id': 1801}],49876,"[{'y': 81, 'x': 68}, {'y': 92, 'x': 76}]",2499719,Pass,1609,1H,1822.188087,85,177959804,1609---2499719,3
8,8,Simple pass,[{'id': 1801}],167145,"[{'y': 92, 'x': 76}, {'y': 88, 'x': 66}]",2499719,Pass,1609,1H,1822.597077,85,177959806,1609---2499719,3
9,8,Simple pass,[{'id': 1802}],370224,"[{'y': 88, 'x': 66}, {'y': 61, 'x': 78}]",2499719,Pass,1609,1H,1824.882555,85,177959808,1609---2499719,2
10,8,Simple pass,"[{'id': 1401}, {'id': 1802}]",8488,"[{'y': 39, 'x': 22}, {'y': 34, 'x': 53}]",2499719,Pass,1631,1H,1825.684410,85,177959775,1631---2499719,1
11,8,Simple pass,[{'id': 1801}],3560,"[{'y': 66, 'x': 47}, {'y': 16, 'x': 55}]",2499719,Pass,1609,1H,1829.843514,85,177959810,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523417,8,Simple pass,[{'id': 1801}],40756,"[{'y': 68, 'x': 41}, {'y': 44, 'x': 48}]",2565927,Pass,675,1H,2664.676718,85,253301558,675---2565927,2
523418,8,Simple pass,[{'id': 1801}],14723,"[{'y': 44, 'x': 48}, {'y': 7, 'x': 68}]",2565927,Pass,675,1H,2667.588461,85,253301559,675---2565927,2
523419,8,Simple pass,[{'id': 1801}],3310,"[{'y': 7, 'x': 68}, {'y': 34, 'x': 60}]",2565927,Pass,675,1H,2671.101604,85,253301561,675---2565927,3
523420,8,Simple pass,[{'id': 1801}],14723,"[{'y': 34, 'x': 60}, {'y': 67, 'x': 70}]",2565927,Pass,675,1H,2671.442237,85,253301563,675---2565927,2


In [25]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [26]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-26-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [27]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [28]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-28-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [29]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
7,8,Simple pass,[{'id': 1801}],49876,"[{'y': 81, 'x': 68}, {'y': 92, 'x': 76}]",2499719,Pass,1609,1H,1822.188087,85,177959804,1609---2499719,3,13.023056,0.741947,Forward Pass
8,8,Simple pass,[{'id': 1801}],167145,"[{'y': 92, 'x': 76}, {'y': 88, 'x': 66}]",2499719,Pass,1609,1H,1822.597077,85,177959806,1609---2499719,3,12.419340,-2.880990,Backward Pass
9,8,Simple pass,[{'id': 1802}],370224,"[{'y': 88, 'x': 66}, {'y': 61, 'x': 78}]",2499719,Pass,1609,1H,1824.882555,85,177959808,1609---2499719,2,25.959969,-0.982794,Side Pass
10,8,Simple pass,"[{'id': 1401}, {'id': 1802}]",8488,"[{'y': 39, 'x': 22}, {'y': 34, 'x': 53}]",2499719,Pass,1631,1H,1825.684410,85,177959775,1631---2499719,1,37.414436,-0.107115,Forward Pass
11,8,Simple pass,[{'id': 1801}],3560,"[{'y': 66, 'x': 47}, {'y': 16, 'x': 55}]",2499719,Pass,1609,1H,1829.843514,85,177959810,1609---2499719,2,41.135872,-1.335251,Side Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523417,8,Simple pass,[{'id': 1801}],40756,"[{'y': 68, 'x': 41}, {'y': 44, 'x': 48}]",2565927,Pass,675,1H,2664.676718,85,253301558,675---2565927,2,20.957099,-1.158386,Side Pass
523418,8,Simple pass,[{'id': 1801}],14723,"[{'y': 44, 'x': 48}, {'y': 7, 'x': 68}]",2565927,Pass,675,1H,2667.588461,85,253301559,675---2565927,2,38.107217,-0.889498,Side Pass
523419,8,Simple pass,[{'id': 1801}],3310,"[{'y': 7, 'x': 68}, {'y': 34, 'x': 60}]",2565927,Pass,675,1H,2671.101604,85,253301561,675---2565927,3,23.637259,1.989021,Side Pass
523420,8,Simple pass,[{'id': 1801}],14723,"[{'y': 34, 'x': 60}, {'y': 67, 'x': 70}]",2565927,Pass,675,1H,2671.442237,85,253301563,675---2565927,2,28.999310,1.144169,Side Pass


In [30]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [31]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
7,8,Simple pass,[{'id': 1801}],49876,"[{'y': 81, 'x': 68}, {'y': 92, 'x': 76}]",2499719,Pass,1609,1H,1822.188087,85,177959804,1609---2499719,3,13.023056,0.741947,Forward Pass
8,8,Simple pass,[{'id': 1801}],167145,"[{'y': 92, 'x': 76}, {'y': 88, 'x': 66}]",2499719,Pass,1609,1H,1822.597077,85,177959806,1609---2499719,3,12.419340,-2.880990,Backward Pass
9,8,Simple pass,[{'id': 1802}],370224,"[{'y': 88, 'x': 66}, {'y': 61, 'x': 78}]",2499719,Pass,1609,1H,1824.882555,85,177959808,1609---2499719,2,25.959969,-0.982794,Side Pass
10,8,Simple pass,"[{'id': 1401}, {'id': 1802}]",8488,"[{'y': 39, 'x': 22}, {'y': 34, 'x': 53}]",2499719,Pass,1631,1H,1825.684410,85,177959775,1631---2499719,1,37.414436,-0.107115,Forward Pass
11,8,Simple pass,[{'id': 1801}],3560,"[{'y': 66, 'x': 47}, {'y': 16, 'x': 55}]",2499719,Pass,1609,1H,1829.843514,85,177959810,1609---2499719,2,41.135872,-1.335251,Side Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523417,8,Simple pass,[{'id': 1801}],40756,"[{'y': 68, 'x': 41}, {'y': 44, 'x': 48}]",2565927,Pass,675,1H,2664.676718,85,253301558,675---2565927,2,20.957099,-1.158386,Side Pass
523418,8,Simple pass,[{'id': 1801}],14723,"[{'y': 44, 'x': 48}, {'y': 7, 'x': 68}]",2565927,Pass,675,1H,2667.588461,85,253301559,675---2565927,2,38.107217,-0.889498,Side Pass
523419,8,Simple pass,[{'id': 1801}],3310,"[{'y': 7, 'x': 68}, {'y': 34, 'x': 60}]",2565927,Pass,675,1H,2671.101604,85,253301561,675---2565927,3,23.637259,1.989021,Side Pass
523420,8,Simple pass,[{'id': 1801}],14723,"[{'y': 34, 'x': 60}, {'y': 67, 'x': 70}]",2565927,Pass,675,1H,2671.442237,85,253301563,675---2565927,2,28.999310,1.144169,Side Pass


In [32]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [33]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-33-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [34]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,1,10,3,10,14,1,10,25,10,14,25,45
10531---2499736,3,4,4,8,7,1,10,7,3,11,16,20
10531---2499741,0,3,4,15,11,3,6,12,6,7,29,24
10531---2499757,0,4,9,10,6,6,4,14,5,13,22,23
10531---2499766,1,5,0,7,13,0,8,5,2,6,20,15
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,1,1,4,6,6,6,11,12,19,6,18,42
756---2565892,5,4,1,15,9,1,19,11,2,10,25,32
756---2565904,4,6,2,13,5,3,21,10,6,12,21,37


## Pass Types

In [35]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [36]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1
uniqueTeamId,,,,,,,,,
10531---2499726,84,0.071429,0.928571,0.071429,0.928571,0.020408,0.979592,0.190476,0.809524
10531---2499736,47,0.170213,0.765957,0.125000,0.750000,0.166667,0.722222,0.190476,0.809524
10531---2499741,60,0.216667,0.750000,0.153846,0.769231,0.230769,0.730769,0.238095,0.761905
10531---2499757,58,0.172414,0.810345,0.200000,0.800000,0.125000,0.833333,0.214286,0.785714
10531---2499766,41,0.317073,0.682927,1.000000,0.000000,0.217391,0.782609,0.375000,0.625000
...,...,...,...,...,...,...,...,...,...
756---2565880,66,0.196970,0.757576,0.344828,0.586207,0.000000,0.947368,0.166667,0.833333
756---2565892,67,0.074627,0.925373,0.500000,0.500000,0.041667,0.958333,0.051282,0.948718
756---2565904,70,0.214286,0.771429,0.272727,0.636364,0.000000,1.000000,0.315789,0.684211


# Pass_Connectivity

In [37]:
from collections import defaultdict
import networkx as nx

In [38]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [39]:
test_dfdf = passes_df.iloc[:1000]

In [40]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [41]:
connectivity_results

[{'1609---2499719': 4.254545454545455, '1631---2499719': 3.5272727272727273},
 {'1625---2499720': 4.709090909090909, '1651---2499720': 1.6181818181818182},
 {'1610---2499721': 4.844444444444444, '1646---2499721': 4.509090909090909},
 {'1673---2499722': 1.8, '1628---2499722': 3.1272727272727274},
 {'1639---2499723': 3.2444444444444445, '1623---2499723': 3.4727272727272727},
 {'1611---2499724': 4.8545454545454545, '1633---2499724': 3.1454545454545455},
 {'1613---2499725': 2.2545454545454544, '1624---2499725': 3.4},
 {'10531---2499726': 4.2727272727272725, '1619---2499726': 3.981818181818182},
 {'1644---2499727': 2.533333333333333, '1612---2499727': 3.7636363636363637},
 {'1659---2499728': 4.127272727272727, '1627---2499728': 1.4181818181818182},
 {'1659---2499729': 3.6222222222222222, '1644---2499729': 2.9272727272727272},
 {'1646---2499730': 4.618181818181818, '1627---2499730': 1.8333333333333333},
 {'1673---2499731': 3.418181818181818, '1613---2499731': 2.9454545454545453},
 {'1651---2

In [42]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [43]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1609---2499719,4.254545
1631---2499719,3.527273
1625---2499720,4.709091
1651---2499720,1.618182
1610---2499721,4.844444
...,...
3158---2576336,3.636364
3164---2576337,4.911111
3204---2576337,4.345455


In [44]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [45]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3,High_pass_Ratio_Zone2,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.037037,0.135802,0.135802,0.691358,84,0.071429,0.928571,0.071429,0.928571,0.020408,...,0.297619,0.119048,0.166667,0.297619,0.535714,NaN,NaN,NaN,NaN,4.272727
10531---2499736,0.000000,0.090909,0.090909,0.818182,47,0.170213,0.765957,0.125000,0.750000,0.166667,...,0.148936,0.063830,0.234043,0.340426,0.425532,1.0,0.000000,1.000000,0.000000,2.709091
10531---2499741,0.052632,0.122807,0.122807,0.701754,60,0.216667,0.750000,0.153846,0.769231,0.230769,...,0.200000,0.100000,0.116667,0.483333,0.400000,2.0,0.000000,1.000000,0.000000,3.088889
10531---2499757,0.054545,0.109091,0.090909,0.745455,58,0.172414,0.810345,0.200000,0.800000,0.125000,...,0.241379,0.086207,0.224138,0.379310,0.396552,3.0,0.000000,1.000000,0.000000,3.311111
10531---2499766,0.000000,0.157895,0.157895,0.684211,41,0.317073,0.682927,1.000000,0.000000,0.217391,...,0.121951,0.048780,0.146341,0.487805,0.365854,1.0,0.000000,1.000000,0.000000,2.844444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.000000,0.238095,0.238095,0.523810,66,0.196970,0.757576,0.344828,0.586207,0.000000,...,0.181818,0.287879,0.090909,0.272727,0.636364,7.0,0.428571,0.285714,0.285714,3.200000
756---2565892,0.015625,0.109375,0.125000,0.750000,67,0.074627,0.925373,0.500000,0.500000,0.041667,...,0.164179,0.029851,0.149254,0.373134,0.477612,2.0,1.000000,0.000000,0.000000,3.044444
756---2565904,0.000000,0.179104,0.164179,0.656716,70,0.214286,0.771429,0.272727,0.636364,0.000000,...,0.142857,0.085714,0.171429,0.300000,0.528571,2.0,0.000000,0.000000,1.000000,4.072727


In [46]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [47]:
half

'1H'

# END OF FILE